# Overview
This project seeks to implement a very simple Dense Neural Network to determine whether a wine is white or red. We'll go through data importing, cleaning, transformation, model building, model training and finally model validation.

# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Input
from keras.optimizers import Adam,RMSprop
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Data Importing

In [ ]:
data = pd.read_csv('/kaggle/input/wine-quality/winequalityN.csv')

# Data exploration

In [ ]:
data.head()

In [ ]:
data.columns.values

# Data cleaning

In [ ]:
cols_to_norm = ['fixed acidity', 'volatile acidity', 'citric acid','residual sugar', 'chlorides', 'free sulfur dioxide','total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

In [ ]:
data['type'].unique()

In [ ]:
for i in range(len(data)):
    if data.iloc[i]['type'] == 'white':
        data.at[i,'type'] = 0.0
    if data.iloc[i]['type'] == 'red':
        data.at[i,'type'] = 1.0

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

# Data Transformation

In [ ]:
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min())/(x.max() - x.min()))

In [ ]:
data.head()

In [ ]:
X = data[cols_to_norm]
X = np.asarray(X)
X[:5]

In [ ]:
y = data['type']
y = np.asarray(y)
from keras.utils import to_categorical
y = to_categorical(y,2)
y[:5]

In [ ]:
X[0].shape

# Data splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)
print(len(X_train),len(X_test))

# Model building

In [ ]:
#Input data shape: (11,)
input_layer = Input(shape=(11,))
dense_layer = Dense(24,activation='relu')(input_layer)
dropout = Dropout(0.5)(dense_layer)
dense_layer = Dense(2,activation='sigmoid')(dropout)
model = keras.Model(input_layer,dense_layer,name='classifier')

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

# Model training

In [ ]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test))

# Model validation

In [ ]:
predictions = model.predict(X_test)
predictions[:5]

In [ ]:
predictions = np.argmax(predictions,axis=1)

In [ ]:
validation = np.argmax(y_test,axis=1)

In [ ]:
conf = confusion_matrix(validation,predictions)

In [ ]:
conf_matrix = pd.DataFrame(conf, index = ['White','Red'],columns = ['White','Red'])
#Normalizing
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize = (18,15))
sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 15})

## Not bad, huh?